## **Домашняя работа 1. Гриневская Катя, БКЛ182**

Сначала импортирую всё необходимое

In [1]:
import urllib.request
import re
import os
import nltk
from nltk.corpus import *
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
! pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 8.0MB 4.2MB/s 


In [4]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Сохраняю две ссылки на положительные и отрицательные отзывы (сразу по 50 штук на странице) на фильм "Чёрная пантера" с Кинопоиска. По 45 отзывов я возьму для составления тонального словаря, по 5 оставлю для проверки

In [5]:
url_p = 'https://www.kinopoisk.ru/film/623250/reviews/ord/rating/status/good/perpage/50'
url_n = 'https://www.kinopoisk.ru/film/623250/reviews/ord/rating/status/bad/perpage/50'

Функция, которая парсит html-страничку и оставляет только список отзывов

In [6]:
def parser(url):
  page = urllib.request.urlopen(url)
  text = page.read().decode('utf-8')
  soup = BeautifulSoup(text, 'html.parser')
  recs = soup.find_all('div', {'class': 'brand_words'}) 
  otzs = []
  # очищаем от остатков html
  for res in recs:
    otzs.append(res.text)
  clean_otzs = []
  for otz in otzs:
    clean_otz = re.sub(r'[\n\t\r]', ' ', otz)
    clean_otzs.append(clean_otz)
  return clean_otzs

In [7]:
clean_otzs_p_all = parser(url_p)

In [8]:
clean_otzs_n_all = parser(url_n)

In [9]:
clean_otzs_p, clean_otzs_p_test = clean_otzs_p_all[:45], clean_otzs_p_all[45:]

In [10]:
clean_otzs_n, clean_otzs_n_test = clean_otzs_n_all[:45], clean_otzs_n_all[45:]

In [11]:
clean_otzs_test = clean_otzs_p_test + clean_otzs_n_test

Проверяю, все ли отзывы скачались

In [12]:
len(clean_otzs_p)

45

In [13]:
len(clean_otzs_n)

45

In [14]:
len(clean_otzs_test)

10

Токенизиурем, лемматизируем, избавляемся от стоп-слов, возвращаем список токенов поданного отзыва

In [15]:
def to_list(otz):
  otz_tokens = []
  numb = re.compile(r'\d')
  tokens = nltk.word_tokenize(otz)
  for token in tokens:
    if token not in """!@#$%^&*{}()_+`’`'?/--–.<>,|«»\"~№=-:;""":
      # проверяем, чтобы не было числом
      # не использую .isalpha(), потому что хочу оставить слова с дефисом
      if len(numb.findall(token)) == 0:
        for char in token:
          if char in """!@#$%^&*{}«»()`_+'?/.<>,|\"~№=:;""":
            token = token.replace(char, '')
        anal = morph.parse(token)
        token_lemm = anal[0].normal_form
        if token_lemm not in russian_stopwords:
          otz_tokens.append(token_lemm)
  # freqlist = dict(Counter(rez_tokens))
  return otz_tokens

Теперь объединяем токены всех отзывов в один список, делаем из него словарь с частотностями, убираем слова с частотностью не больше 3

In [16]:
def to_dict(clean_otzs):
  rez_tokens = []
  rez_tokens_d = {}
  for otz in clean_otzs:
    rez_tokens += to_list(otz)
  freqlist = dict(Counter(rez_tokens))
  for key, value in freqlist.items():
    if value > 3:
      rez_tokens_d[key] = value
  return rez_tokens_d

In [17]:
rez_tokens_d_p = to_dict(clean_otzs_p)

In [18]:
rez_tokens_d_n = to_dict(clean_otzs_n)

Оставляем в одном из полученных множеств те слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Получаем словарь {'pos' : {'token': frequency, ...}, 'neg': {'token': frequency, ...}}

In [20]:
def clean_dicts(dict_pos, dict_neg):
  dict_pos_uniq = {}
  dict_neg_uniq = {}
  for key in dict_neg.keys():
    if key not in dict_pos.keys():
      dict_neg_uniq[key] = dict_neg[key]
  for key in dict_pos.keys():
    if key not in dict_neg.keys():
      dict_pos_uniq[key] = dict_pos[key]
  freq_dict = {'pos': dict_pos_uniq, 'neg' : dict_neg_uniq}
  return freq_dict

In [21]:
freq_dict = clean_dicts(rez_tokens_d_p, rez_tokens_d_n)

Теперь у нас есть тональный словарь. Создаём функцию, которая проверяет, сколько общих слов у проверяемого отзыва со словарём из положительных отзывов, а сколько - со словарём из отрицательных.

In [23]:
def tone_detect(freq_dict, test_text):
  counts = Counter()
  test_text_tokens = to_list(test_text)
  for tone, freqdict in freq_dict.items():
    for word in test_text_tokens:
      if word in freqdict.keys():
        counts[tone] += freqdict[word]
  return counts.most_common()

Прогоняем на всех 10 отзывах, оставленных для проверки

In [33]:
for text in clean_otzs_test:
  print(tone_detect(freq_dict, text)[0][0])

pos
pos
pos
pos
pos
neg
pos
neg
neg
pos


Итого: из 10 текстов на проверку 5 первых были позитивными, 5 вторых - негативными. Все позитивные отзывы правильно определены, а негативные - не все. Я думаю, что это связано с тем, что в негативных отзывах часто могут упоминаться словосочетания типа "не впечатлило", "не то чтобы прелестный", когда автор использует редкое позитивное слово с отрицательными единицами, которые уходят засчёт стоп-слов или удаления совпадающих элементов в тональных словарях.

Теперь посчитаем accuracy

In [27]:
!pip install sklearn

In [28]:
from sklearn.metrics import accuracy_score

In [34]:
results = []
gold = ['pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg']
for text in clean_otzs_test:
  results.append(tone_detect(freq_dict, text)[0][0])
print('RESULTS:')
print('Accuracy: %.4f' % accuracy_score(results, gold))

RESULTS:
Accuracy: 0.8000


Улучшить программу можно следующими способами: 

1.   выбрать другой способ обкачки сайта с отзывами, чтобы можно было за раз получить большее количество отзывов, чтобы увеличить объём тональных словарей
2.   придумать более точный вариант сравнения частотностей слов с частотностями из тональных словарей: например, не просто складывать количество упоминаний, а складывать именно частотности (количество упоминаний делённое на объём текста в случае с отзывом на проверку или на сумму объёмов текстов в случае с тональными словарями)
3. ещё можно получше чистить от стоп-слов и шума (например, загрузить откуда-нибудь более полный список стоп-слов для русского языка, хотя они при применении п.1 должны уйти при взаимоисключении элементов из двух тональных словарей)